In [1]:
from simulation.notebook_utils import *
from simulation.utils import *

import busybee

In [2]:
# [!] CHANGE `24` to `1` for shorter simulations
SIM_TIME_MS = 24 * 3_600 * 1_000

#
# Global config for both ONLINE and OFFLINE
#

DEFAULT_CONFIG = NotebookSimulationConfig(
    loopix_kwargs= {
        'user_rate_pull': 1,
        'user_rate_payload': 2,
        'user_rate_drop': 2,
        'user_rate_loop': 2,
        'user_rate_delay': 3,
        'mix_rate_loop': 2,
        'mix_rate_loop_delay':3,
    },
    app_kwargs= {
        'init_rate_per_second': 1/30,
        'multi_message': 1,
        'heavy_user_percentage': 20,
        'heavy_user_weight': 4,
    }
)

SLOW_RATE = 1/30

CONFIG_NAMES_TO_CONFIGS = {
    'rs': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 1,
        }),
    'baseline': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'round_multiplicator': 5,
        },
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 1,
        }),
    'ps-2': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 1.6668563120298376,
            'user_rate_drop': 1.6668563120298376,
            'user_rate_loop': 1.6668563120298376,
        }),
    'ps-5': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 1.3227983043620035,
            'user_rate_drop': 1.3227983043620035,
            'user_rate_loop': 1.3227983043620035,
        }),
}

In [3]:
DEFAULT_SCHEDULES = ['gen_os10', 'gen_os15', 'gen_os20']

all_schedules = {k: read_compressed_bool_schedules("input/schedules_%s.txt.gz" % k) for k in DEFAULT_SCHEDULES}
all_schedules['online'] = None
print("Loaded schedules")

Loaded schedules


In [4]:
def pickle_to_file(args):
    pickle_down({args[0]: args[1]})

def create_sims(groupsizes, configs, schedules, strategies):
    sims = dict()
    for m, group_sizes in groupsizes:

        for config_name in configs:
            config = CONFIG_NAMES_TO_CONFIGS[config_name]

            for schedule_name in schedules:
                schedule = all_schedules[schedule_name]

                for strategy_name in strategies:
                        sim = create_scenario(m, group_sizes, config, strategy_name, SIM_TIME_MS, schedule if schedule else [])
                        name = get_name_for_sim(sim, config_name, schedule_name, strategy_name)

                        sims[name] = sim
                        sim.name = name.replace('.input', '').upper()


    _ = busybee.map(pickle_to_file, [(k, v) for k, v in sims.items()], processes=4)
    print("Saved %d files" % len(sims))

In [5]:
#
# ONLINE scenarios
#

ONLINE_GROUP_SIZES = [
    (25, [25]),
    (32, [32]),
    (45, [45]),
    (64, [64]),
    (91, [91]),
    (128, [128]),
    (181, [181]),
    (256, [256]),
]
ONLINE_CONFIGS = ['rs']
ONLINE_SCHEDULES = ['online']


create_sims(ONLINE_GROUP_SIZES, ['rs'], ONLINE_SCHEDULES, ['unicast', 'rollercoaster-k1-p1-notimeout', 'rollercoaster-k2-p2-notimeout'])
create_sims(ONLINE_GROUP_SIZES, ['ps-2'], ONLINE_SCHEDULES, ['polysphinx-p2-l1'])
create_sims(ONLINE_GROUP_SIZES, ['ps-5'], ONLINE_SCHEDULES, ['polysphinx-p5-l1'])
create_sims(ONLINE_GROUP_SIZES, ['baseline'], ONLINE_SCHEDULES, ['unicast'])

BusyBee: Start processing 24 items with 4 processes...
BusyBee: 12/24, 50.0% (avg: 0ms cpu, rem: 0.0s)
BusyBee: Finished processing 24 items in 0.0s (avg: 0ms cpu)
Saved 24 files
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
BusyBee: Start processing 8 items with 4 processes...
BusyBee: 4/8, 50.0% (avg: 0ms cpu, rem: 0.0s)
BusyBee: Finished processing 8 items in 0.0s (avg: 0ms cpu)
Saved 8 files
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: lambda/mu > 2
For secure configuration it should hold that: l